In [1]:
from delierium.Infinitesimals import prolongationODE
from delierium.JanetBasis import Janet_Basis
from pylatexenc.latexwalker import LatexWalker
x = var('x')
y=function('y')
ode=4*diff(y(x),x,x)*y(x) - Rational(3/4)* (diff(y(x),x)**2)-12*(y(x)**3)
p=prolongationODE(ode, y, x)[0].expression()
print(p)

-4*y(x)*D[0, 0](xi)(y(x), x)*diff(y(x), x)^3 + 36*xi(y(x), x)*y(x)^2*diff(y(x), x) + 4*y(x)*D[0, 0](phi)(y(x), x)*diff(y(x), x)^2 - 8*y(x)*D[0, 1](xi)(y(x), x)*diff(y(x), x)^2 + 3/2*D[0](xi)(y(x), x)*diff(y(x), x)^3 - 12*y(x)*D[0](xi)(y(x), x)*diff(y(x), x)*diff(y(x), x, x) - 36*phi(y(x), x)*y(x)^2 + 8*y(x)*D[0, 1](phi)(y(x), x)*diff(y(x), x) - 4*y(x)*D[1, 1](xi)(y(x), x)*diff(y(x), x) - 3/2*D[0](phi)(y(x), x)*diff(y(x), x)^2 + 3/2*D[1](xi)(y(x), x)*diff(y(x), x)^2 + 4*y(x)*D[0](phi)(y(x), x)*diff(y(x), x, x) - 8*y(x)*D[1](xi)(y(x), x)*diff(y(x), x, x) - 5/2*xi(y(x), x)*diff(y(x), x)*diff(y(x), x, x) - 4*xi(y(x), x)*y(x)*diff(y(x), x, x, x) - 1/2*(72*y(x)^2*diff(y(x), x) - 5*diff(y(x), x)*diff(y(x), x, x) - 8*y(x)*diff(y(x), x, x, x))*xi(y(x), x) + 4*y(x)*D[1, 1](phi)(y(x), x) - 3/2*D[1](phi)(y(x), x)*diff(y(x), x) + 4*phi(y(x), x)*diff(y(x), x, x)


In [2]:
ops = p.operands()

In [3]:
print(ops)

[-4*y(x)*D[0, 0](xi)(y(x), x)*diff(y(x), x)^3, 36*xi(y(x), x)*y(x)^2*diff(y(x), x), 4*y(x)*D[0, 0](phi)(y(x), x)*diff(y(x), x)^2, -8*y(x)*D[0, 1](xi)(y(x), x)*diff(y(x), x)^2, 3/2*D[0](xi)(y(x), x)*diff(y(x), x)^3, -12*y(x)*D[0](xi)(y(x), x)*diff(y(x), x)*diff(y(x), x, x), -36*phi(y(x), x)*y(x)^2, 8*y(x)*D[0, 1](phi)(y(x), x)*diff(y(x), x), -4*y(x)*D[1, 1](xi)(y(x), x)*diff(y(x), x), -3/2*D[0](phi)(y(x), x)*diff(y(x), x)^2, 3/2*D[1](xi)(y(x), x)*diff(y(x), x)^2, 4*y(x)*D[0](phi)(y(x), x)*diff(y(x), x, x), -8*y(x)*D[1](xi)(y(x), x)*diff(y(x), x, x), -5/2*xi(y(x), x)*diff(y(x), x)*diff(y(x), x, x), -4*xi(y(x), x)*y(x)*diff(y(x), x, x, x), -1/2*(72*y(x)^2*diff(y(x), x) - 5*diff(y(x), x)*diff(y(x), x, x) - 8*y(x)*diff(y(x), x, x, x))*xi(y(x), x), 4*y(x)*D[1, 1](phi)(y(x), x), -3/2*D[1](phi)(y(x), x)*diff(y(x), x), 4*phi(y(x), x)*diff(y(x), x, x)]


In [22]:
from IPython.core.debugger import set_trace
import re
re_num   = re.compile(r"^(?P<sign>[+-])?(?P<p1>\d*)?(?P<p2>[.|/]\d*)?$")
re_diff1 = re.compile(r"^D\[(?P<vars>.+)\]\((?P<f1>[^\)]+)\)\((?P<args>.*)\)(\^(?P<exp>.+)$)?")
re_diff2 = re.compile(r"^diff\((?P<diff>.+)\)(\^(?P<exp>.+)$)?")                      
nakedf   = re.compile(r"^(?P<fname>\w+)\(.*$")
latexf   = re.compile(r"^(?P<f>(\\)?\w+)")

def latexer(e):
    ops = e.expand().operands()
    def _latexer(t):
        o = t.operands()
        res = ""
        for _o in o: 
            if match := re_num.match(str(_o)):
                gd   = match.groupdict()
                sign = gd["sign"]
                if not sign:
                    sign = ""
                p1   = gd["p1"] if gd["p1"] else 0
                p2   = gd["p2"] if gd["p2"] else ""
                res  = sign + str(p1) + str(p2) + res
            elif match := re_diff2.match(str(_o)):
                params = match.groupdict()["diff"].split(",")
                params = [_.strip() for _ in params]
                fu = params[0]
                if not match.groupdict().get("exp", ""):
                    res += " %s_{%s}" % (nakedf.match(fu).groupdict()["fname"], "".join(_ for _ in params[1:]))
                else:
                    res += " %s_{%s}^%s" % (nakedf.match(fu).groupdict()["fname"], "".join(_ for _ in params[1:])
                                       , match.groupdict()["exp"])
            elif match := re_diff1.match(str(_o)):
                params  = match.groupdict()["args"].split(",")
                params  = [_.strip() for _ in params]
                fu      = params[0]
                vv      = [int(_) for _ in match.groupdict()["vars"].split(",")]
                params  = [nakedf.match(fu).groupdict()["fname"]] + params[1:]    
                latexf1 = re.compile(r"^.+\\left\((?P<f>(\\)?%s)" % match.groupdict()["f1"])
                fn      = latexf1.match(_o._latex_()).groupdict()["f"]
                if not match.groupdict().get("exp", ""):                    
                    res += r" %s_{%s}" % (fn, "".join((params[i] for i in vv)))
                else:
                    res += r" %s_{%s}^%s" % (fn, 
                                       "".join((params[i] for i in vv))), match.groupdict()["exp"]
            elif match := nakedf.match(str(_o)):
                n = match.groupdict()["fname"]
                lf = _o._latex_()    
                res += r" %s" % latexf.match(lf).groupdict()["f"]
        return res
    
    all = ""
    for _ in ops:
        r = _latexer(_)
        
        if r.startswith("-"):
            all += r
        else:
            all += "+"+r
    return html("<p>$"+all+"$</p>")
latexer(p)

<p>\(-4 y \xi_{yy} y_{x}^3+4 y \phi_{yy} y_{x}^2-8 y \xi_{yx} y_{x}^2+3/2 \xi_{y} y_{x}^3-12 y \xi_{y} y_{x} y_{xx}-36 \phi y+8 y \phi_{yx} y_{x}-4 y \xi_{xx} y_{x}-3/2 \phi_{y} y_{x}^2+3/2 \xi_{x} y_{x}^2+4 y \phi_{y} y_{xx}-8 y \xi_{x} y_{xx}+4 y \phi_{xx}-3/2 \phi_{x} y_{x}+4 \phi y_{xx}\)</p>